In [ ]:
import sys

sys.path.insert(1, "../")

import rasterio as rio
import numpy as np
from shapely import from_wkt
from dem_handler.utils.spatial import transform_polygon
from dem_comparison.analysis import analyse_difference_for_interval
from dem_comparison.utils import simple_mosaic
from dem_comparison.utils import get_cross_section_data, enhance_image
from dem_comparison.plots import plot_cross_sections
from pathlib import Path
import os
import glob
from dem_comparison.utils import reproject_match_tifs
from dem_handler.utils.rio_tools import reproject_profile_to_new_crs

In [ ]:
poly = "POLYGON ((-1478972.5805124296 -584351.9503323506, -1429491.2228967457 -609735.2441741625, -1466227.382338693 -677424.0277523277, -1514959.0224147453 -653861.4765067638, -1478972.5805124296 -584351.9503323506))"
AOI_1_Bounds = transform_polygon(from_wkt(poly), 3031, 4326).bounds
AOI_1_Bounds = (
    *(np.floor(AOI_1_Bounds[0:2]).astype("int")).tolist(),
    *(np.ceil(AOI_1_Bounds[2:]).astype("int")).tolist(),
)
lat_range = range(AOI_1_Bounds[1], AOI_1_Bounds[3])
lon_range = range(AOI_1_Bounds[0], AOI_1_Bounds[2])
lat_range, lon_range

In [ ]:
location_name = "Artifact_1"
analyse_difference_for_interval(
    lat_range,
    lon_range,
    temp_path=Path(f"TEMP_{location_name}"),
    save_dir_path=Path(f"{location_name}_Outputs"),
    use_multiprocessing=True,
    query_num_tasks=None,
    keep_temp_files=True,
    return_outputs=False,
    num_cpus=4,
    rema_index_path=Path("../dem_comparison/data/REMA_Mosaic_Index_v2.gpkg"),
    cop30_index_path=Path("../dem_comparison/data/copdem_tindex_filename.gpkg"),
)

In [ ]:
os.makedirs(f"{location_name}_Outputs/{location_name}_mosaics", exist_ok=True)
diff_arrays = [Path(p) for p in glob.glob(f"{location_name}_Outputs/dem_diff/*.tif")]
diff_mos = Path(f"{location_name}_Outputs/{location_name}_mosaics/diff_mos.tif")
if diff_mos.exists():
    os.remove(diff_mos)
simple_mosaic(diff_arrays, diff_mos, force_bounds=from_wkt(poly).bounds)

In [ ]:
with rio.open(diff_mos, "r") as ds:
    tr = ds.transform
    rema_resolution = (tr.a, -tr.e)


cop_dems = glob.glob(f"TEMP_{location_name}/**/TEMP_COP_ELLIPSOID.tif")
rema_dems = glob.glob(f"TEMP_{location_name}/**/TEMP_REMA.tif")

cop_mosaic = Path(f"{location_name}_Outputs/cop_mosaic.tif")
rema_mosaic = Path(f"{location_name}_Outputs/rema_mosaic.tif")

simple_mosaic(
    cop_dems,
    cop_mosaic,
    force_bounds=transform_polygon(from_wkt(poly), 3031, 4326).bounds,
)
simple_mosaic(
    rema_dems,
    rema_mosaic,
    force_bounds=from_wkt(poly).bounds,
    resolution=rema_resolution,
)
matched_cop_mosaic = Path(
    f"{location_name}_Outputs/{location_name}_mosaics/matched_cop_mosaic.tif"
)
matched_rema_mosaic = Path(
    f"{location_name}_Outputs/{location_name}_mosaics/matched_rema_mosaic.tif"
)
reproject_match_tifs(
    rema_mosaic,
    cop_mosaic,
    target_crs=3031,
    verbose=True,
    convert_dB=False,
    save_path_1=matched_rema_mosaic,
    save_path_2=matched_cop_mosaic,
)
os.remove(cop_mosaic)
os.remove(rema_mosaic)

In [ ]:
rasters = [matched_cop_mosaic, matched_rema_mosaic]
plot_cross_sections(
    rasters,
    from_wkt(poly),
    diff_mos,
    raster_names=["COP", "REMA"],
    raster_colours=["orange", "cyan"],
    save_path="cross_plot.html",
)